# 44. MNIST CNN with Ensemble

<p style="text-align: right;">
blackdew7@gmail.com<br>
Your name :
</p>

#### 선행지식
1. TensorFlow 다루기 기초
2. 모델링을 한다는 것에 대한 이해.
3. Supervised Learning 중 Classification에 대한 기본 지식.
4. CNN에 대한 기본 구조와 개념

#### 실습목표
1. CNN의 구조를 Graph로 그려낼 수 있다.
2. 그려낸 Graph를 텐서플로우를 이용해 코딩할 수 있다.
3. Dropout을 이해하고 사용할 수 있다.
4. Ensemble을 이해하고 사용할 수 있다.
5. CNN 레이어 구조를 Class로 구현할 수 있다.

#### 사용데이터.

01. Multinomial Classification : http://yann.lecun.com/exdb/mnist/

## 00. 라이브러리 불러오기

In [0]:
# 메모리 초기화
%reset

In [0]:
# MNIST and Convolutional Neural Network
import numpy as np
import tensorflow as tf
import random

## 01. 데이터 불러오기 & 전처리

In [0]:
'''
tf.contrib.learn.datasets.load_dataset() 함수의 사용
'''

mnist = tf.contrib.learn.datasets.load_dataset("mnist")
print(mnist.train.images.shape)

test_input, test_label = mnist.train.next_batch(500)
print(test_input.shape)
print(test_label.shape)

# label이 (500,) -> (500, 10) 형태가 되어야 해서 단위행렬로 변환해 주는 과정이 필요하다.
print(np.eye(10)[test_label].shape)

## 02. Class 만들기

In [0]:
# Graph Clear
tf.reset_default_graph()

class ModelCNN:
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        
        self.prepare()
        self.build_net()
        
    def prepare(self):
        self.X = tf.placeholder(tf.float32, [None, 28, 28, 1])
        self.Y = tf.placeholder(tf.float32, [None, 10])
        self.kp = tf.placeholder(tf.float32)
        self.lr = tf.placeholder(tf.float32)
    
    def build_net(self):        
        conv1 = self.conv(self.X, 1, 32)
        conv2 = self.conv(conv1, 32, 64)
        flat = tf.reshape(conv2, [-1, 7 * 7 * 64])
        fc = self.fc(flat, 7 * 7 * 64, 1000, activation=True, dropout=True)
        
        self.logits = self.fc(fc, 1000, 10, activation=False, dropout=False)
        self.cost = self.cost(self.logits, self.Y)
        self.accuracy = self.accuracy(self.logits, self.Y)

        self.optimizer = self.optimizer(self.cost)
        
    def conv(self, x, in_cnt, out_cnt):
        # for Conv Layer filter - shape=(3, 3, 1)
        w = tf.Variable(tf.random_normal([3, 3, in_cnt, out_cnt], stddev=0.01))
        b = tf.Variable(tf.random_normal([out_cnt], stddev=0.01))

        # Convolution Layer
        conv = tf.nn.conv2d(x, w, strides=[1, 1, 1, 1], padding='SAME')
        conv = tf.add(conv, b)
        conv = tf.nn.relu(conv)

        # Pooling Layer
        pool = tf.nn.max_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        
        return pool
    
    def fc(self, x, in_cnt, out_cnt, activation=True, dropout=True):
        # for Final FC
        W = tf.Variable(tf.random_normal([in_cnt, out_cnt], stddev=0.01))
        b = tf.Variable(tf.random_normal([out_cnt], stddev=0.01))

        # for Final FC
        fc = tf.add(tf.matmul(x, W), b)
        
        # activation
        if (activation):
            fc = tf.nn.relu(fc)
            
        # dropout
        if (dropout):
            fc = tf.nn.dropout(fc, keep_prob=self.kp)
        
        return fc
    
    def cost(self, logits, Y):
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y)
        cost = tf.reduce_mean(cross_entropy)
        return cost
    
    def optimizer(self, cost):
        optimizer = tf.train.AdamOptimizer(learning_rate=self.lr).minimize(cost)
        return optimizer

    def accuracy(self, logits, Y):
        correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        return accuracy

    def train(self, x_data, y_data, prob=0.7, lr=0.01):
        return self.sess.run(self.optimizer, feed_dict={self.X: x_data, self.Y: y_data, self.kp: prob, self.lr: lr})

    def predict(self, x_test, prob=1.0, lr=0.01):
        return self.sess.run(self.logits, feed_dict={self.X: x_test, self.kp: prob, self.lr: lr})

    def get_cost(self, x_data, y_data, prob=0.7, lr=0.01):
        return self.sess.run(self.cost, feed_dict={self.X: x_data, self.Y: y_data, self.kp: prob, self.lr: lr})
        
    def get_accuracy(self, x_test, y_test, prob=1.0, lr=0.01):
        return self.sess.run(self.accuracy, feed_dict={self.X: x_test, self.Y: y_test, self.kp: prob, self.lr: lr})

## 04. Ensemble

In [0]:
# initialize
sess = tf.Session()

models = []
num_models = 5
for m in range(num_models):
    models.append(ModelCNN(sess, "model" + str(m)))

sess.run(tf.global_variables_initializer())

print('Learning Started!')

epochs = 3
batch_size = 256
n_of_batches = int(mnist.train.num_examples / batch_size)

# train each model
for m_idx, m in enumerate(models):
    print("%dth models" % (m_idx + 1))
    
    for epoch in range(epochs):
        print("%dth epoch" % (epoch + 1))
        
        for i in range(n_of_batches):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            m.train(batch_xs.reshape([-1, 28, 28, 1]), np.eye(10)[batch_ys])

            # 학습 상황 디스플레이
            if ((i + 1) % 10 == 0):
                loss = m.get_cost(batch_xs.reshape([-1, 28, 28, 1]), np.eye(10)[batch_ys])
                print("%dth records, training cost: %.3f" % (((i + 1) * batch_size), loss))

        print('Accuracy:', m.get_accuracy(mnist.test.images.reshape([-1, 28, 28, 1]), np.eye(10)[mnist.test.labels]))
    
print('Learning Finished!')


In [0]:
# Test model and check accuracy
test_size = len(mnist.test.labels)
predictions = np.zeros([test_size, 10])
for m_idx, m in enumerate(models):
    print((m_idx + 1), 'Accuracy:', m.get_accuracy(mnist.test.images.reshape([-1, 28, 28, 1]), np.eye(10)[mnist.test.labels]))
    p = m.predict(mnist.test.images.reshape([-1, 28, 28, 1]))
    predictions += p
    
print(predictions)
print(tf.argmax(predictions, 1))

# ensemble accuracy
ensemble_correct_prediction = tf.equal(tf.argmax(predictions, 1), mnist.test.labels)
ensemble_accuracy = tf.reduce_mean(tf.cast(ensemble_correct_prediction, tf.float32))

print('Ensemble accuracy:', sess.run(ensemble_accuracy))

### 성능을 한 번 확인해 보자. 제대로 되고 있는걸까?

In [0]:
# Get one and predict
r = random.randint(0, mnist.test.num_examples - 1)
predictions = np.zeros([1, 10])
for m_idx, m in enumerate(models):
    p = m.predict(mnist.test.images[r:r + 1].reshape([-1, 28, 28, 1]))
    predictions += p
    
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
print("Prediction: ", sess.run(tf.argmax(predictions, 1)))

import matplotlib.pyplot as plt
plt.imshow(mnist.test.images[r:r + 1].reshape(28, 28), cmap='Greys', interpolation='nearest')
plt.show()

In [0]:
# 세션을 종료하자
sess.close()